In [20]:
import pandas as pd
import os 
os.chdir(r'./')
import itertools
import math
df_Votes = pd.read_csv('BAF-al_2020.csv',dtype={'GEOID20_left':str,'COUNTYFP20':str})

''' Uncomment read_csv file to generate results 
     comapring current Alabama VRA district to the 2 
     proposed in Whole County Map '''

df_CD = pd.read_csv('AL_94_CD116_7.csv', dtype={'GEOID':str,'VTD':str,'CD116':str,'COUNTY':str})
#df_CD = pd.read_csv('AL_94_WC_6.csv', dtype={'GEOID':str,'VTD':str,'CD116':str,'COUNTY':str})
#df_CD = pd.read_csv('AL_94_WC_7.csv', dtype={'GEOID':str,'VTD':str,'CD116':str,'COUNTY':str})

In [21]:
df_Votes = df_Votes[['VTDST20','COUNTYFP20','G20PRERTRU','G20PREDBID']]
df_Votes = df_Votes.drop_duplicates().reset_index(drop=True)
df_Votes = df_Votes.rename(columns={'VTDST20':'VTD','COUNTYFP20':'COUNTY'})

df_CD = df_CD[['VTD','COUNTY','P0010001','P0010004']]
df_CD = df_CD.groupby(['VTD','COUNTY']).sum()
df = df_CD.merge(df_Votes, how='inner',on=['VTD','COUNTY'])
df = df.drop_duplicates()
df = df.drop(columns=['COUNTY'])
df_sum = df.groupby(['VTD']).sum().reset_index()
df = df[['VTD']].merge(df_sum, on='VTD')
df = df.drop_duplicates()
assert len(df.index) != 0
assert len(df_Votes.index) != 0

In [22]:
df['Black_Percentage'] = df['P0010004']/df['P0010001']
df['Group'] = None
df = df.reset_index(drop=True)

for i ,row in df.iterrows():
    if row['Black_Percentage'] >= .9:
        df['Group'].iloc[i] = 'B90%'
    if row['Black_Percentage'] < .9:
        df['Group'].iloc[i] = 'B60%'
    if row['Black_Percentage'] < .6:
        df['Group'].iloc[i] = 'N50%'
    if row['Black_Percentage'] <= .4:
        df['Group'].iloc[i] = 'W60%'
    if row['Black_Percentage'] <= .1:
        df['Group'].iloc[i] = 'W90%'

quoat = 0
quoat = df[['G20PRERTRU','G20PREDBID']].sum()
quoat = (quoat['G20PREDBID'] + quoat['G20PRERTRU'])/2
voteGroups = df[['Group','G20PREDBID']].groupby('Group').sum().reset_index()
voteGroups = voteGroups.values.tolist()

c:\Miniconda\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [27]:
output = {'B90%':0,'B60%':0,'W90%':0,'W60%':0,'N50%':0}
for i in itertools.permutations(voteGroups):
    c = 0
    for j in i:
        c += j[1]
        if c > quoat:
            output[j[0]] += 1
            break
df_share = pd.DataFrame(list(output.items()),columns = ['Cat','S_Count'])
factorial = math.factorial(len(output.keys()))
df_share['S_Index'] = df_share['S_Count']/factorial

In [28]:
df_share

,Cat,S_Count,S_Index
0,B90%,50,0.416667
1,B60%,50,0.416667
2,W90%,0,0.000000
3,W60%,10,0.083333
4,N50%,10,0.083333
